In [3]:
!pip install psycopg2

In [1]:
import psycopg2
import pandas as pd
from psycopg2 import sql

In [2]:
password = ""
with open('password.txt', 'r') as f:
    password = f.readline().strip()
#print(password)
db_name = "nyc_taxi_adv"

In [3]:
conn = psycopg2.connect(
    dbname="postgres", 
    user="postgres", 
    password= password, 
    host="localhost", 
    port="5432"
)

cur = conn.cursor()

In [4]:
conn.autocommit = True
#deleting database if exists
drop_query = sql.SQL("DROP DATABASE IF EXISTS {}").format(sql.Identifier(db_name))
cur.execute(drop_query)

In [5]:
#adding database if exists
create_query = sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name))
cur.execute(create_query)
#conn.commit()

In [6]:
cur.close()
conn.close()

In [7]:
#create table taxi
conn = psycopg2.connect(
    dbname=db_name, 
    user="postgres", 
    password= password, 
    host="localhost", 
    port="5432"
)

cur = conn.cursor()

In [8]:
cur.execute("""CREATE TABLE taxi (
                    id SERIAL PRIMARY KEY,
                    VendorID INT,
                    tpep_pickup_datetime TIMESTAMP,
                    tpep_dropoff_datetime TIMESTAMP,
                    passenger_count INT,
                    trip_distance FLOAT,
                    pickup_longitude FLOAT,
                    pickup_latitude FLOAT,
                    RateCodeID INT,
                    store_and_fwd_flag BOOLEAN,
                    dropoff_longitude FLOAT,
                    dropoff_latitude FLOAT,
                    payment_type INT,
                    fare_amount FLOAT,
                    extra FLOAT,
                    mta_tax FLOAT,
                    tip_amount FLOAT,
                    tolls_amount FLOAT,
                    improvement_surcharge FLOAT,
                    total_amount FLOAT
);""")

In [9]:
conn.commit()

In [10]:
cur.close()
conn.close()

In [11]:
csv_files = ["./yellow_tripdata_2016-01.csv", "./yellow_tripdata_2016-02.csv", "./yellow_tripdata_2016-03.csv"]

In [13]:
#load table taxi
conn = psycopg2.connect(
    dbname=db_name, 
    user="postgres", 
    password= password, 
    host="localhost", 
    port="5432"
)

cur = conn.cursor()

In [14]:
for csv_file in csv_files:
    with open(csv_file, 'r', encoding='utf-8') as f:
        cur.copy_expert("""COPY taxi(VendorID,
                                    tpep_pickup_datetime,
                                    tpep_dropoff_datetime,
                                    passenger_count,
                                    trip_distance,
                                    pickup_longitude,
                                    pickup_latitude,
                                    RateCodeID,
                                    store_and_fwd_flag,
                                    dropoff_longitude,
                                    dropoff_latitude,
                                    payment_type,
                                    fare_amount,
                                    extra,
                                    mta_tax,
                                    tip_amount,
                                    tolls_amount,
                                    improvement_surcharge,
                                    total_amount) 
                            FROM STDIN WITH CSV HEADER""", f)
    print(f"Učitani podaci iz: {csv_file}")


Učitani podaci iz: ./yellow_tripdata_2016-01.csv
Učitani podaci iz: ./yellow_tripdata_2016-02.csv
Učitani podaci iz: ./yellow_tripdata_2016-03.csv


In [17]:
conn.commit()

InterfaceError: connection already closed

In [16]:
cur.close()
conn.close()

In [25]:
#drop columns from taxi
conn = psycopg2.connect(
    dbname=db_name, 
    user="postgres", 
    password= password, 
    host="localhost", 
    port="5432"
)

cur = conn.cursor()

In [26]:
cur.execute("""ALTER TABLE taxi 
            DROP COLUMN VendorID,
            DROP COLUMN RateCodeID,
            DROP COLUMN store_and_fwd_flag,
            DROP COLUMN payment_type,
            DROP COLUMN fare_amount,
            DROP COLUMN extra,
            DROP COLUMN mta_tax,
            DROP COLUMN tolls_amount,
            DROP COLUMN improvement_surcharge;""")

In [27]:
conn.commit()

In [28]:
cur.close()
conn.close()

In [29]:
#adding postgis ekstenzije
conn = psycopg2.connect(
    dbname=db_name, 
    user="postgres", 
    password= password, 
    host="localhost", 
    port="5432"
)

cur = conn.cursor()

In [30]:
cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;");

In [31]:
conn.commit()

In [32]:
cur.close()
conn.close()

In [33]:
#adding geom columns
conn = psycopg2.connect(
    dbname=db_name, 
    user="postgres", 
    password= password, 
    host="localhost", 
    port="5432"
)

cur = conn.cursor()

In [34]:
cur.execute("""
    ALTER TABLE taxi
    ADD COLUMN geom_pickup GEOMETRY(Point, 4326);
""")

In [35]:
cur.execute("""
    ALTER TABLE taxi
    ADD COLUMN geom_dropoff GEOMETRY(Point, 4326);
""")

In [36]:
conn.commit()

In [37]:
cur.close()
conn.close()

In [38]:
#load geom columns
conn = psycopg2.connect(
    dbname=db_name, 
    user="postgres", 
    password= password, 
    host="localhost", 
    port="5432"
)

cur = conn.cursor()

In [39]:
cur.execute("""
    UPDATE taxi
    SET geom_pickup = ST_SetSRID(ST_MakePoint(pickup_longitude, pickup_latitude), 4326);
""")

In [40]:
conn.commit()

In [41]:
cur.execute("""
    UPDATE taxi
    SET geom_dropoff = ST_SetSRID(ST_MakePoint(dropoff_longitude, dropoff_latitude), 4326);
""")

In [42]:
conn.commit()

In [43]:
cur.close()
conn.close()

In [49]:
#create index
conn = psycopg2.connect(
    dbname=db_name, 
    user="postgres", 
    password= password, 
    host="localhost", 
    port="5432"
)

cur = conn.cursor()

In [50]:
cur.execute("""
    CREATE INDEX geom_pickup_idx ON taxi USING GIST (geom_pickup);
""")

In [51]:
cur.execute("""
    CREATE INDEX geom_dropoff_idx ON taxi USING GIST (geom_dropoff);
""")

In [52]:
conn.commit()

In [53]:
cur.close()
conn.close()

In [55]:
#create .shp 

In [56]:
#shp_file = "./taxi_zones/taxi_zones.shp"

In [63]:
f"shp2pgsql -I -s 4326 -c {shp_file} public.taxi_zones | psql -U postgres -d {db_name}"

In [ ]:
#"C:\Program Files\PostgreSQL\17\bin\shp2pgsql.exe" -I -s 2263 -c "C:\Users\User\Desktop\ADV\taxi_zones\taxi_zones.shp" public.taxi_zones | "C:\Program Files\PostgreSQL\17\bin\psql.exe" -U postgres -d nyc_taxi_adv


In [3]:
#create indexies tpep_pickup_datetime
conn = psycopg2.connect(
    dbname=db_name, 
    user="postgres", 
    password= password, 
    host="localhost", 
    port="5432"
)

cur = conn.cursor()

In [4]:
cur.execute("""
    CREATE INDEX tpep_pickup_datetime_idx ON taxi (tpep_pickup_datetime);
""")

In [5]:
conn.commit()

In [6]:
cur.close()
conn.close()

In [ ]:
#create table events
#conn = psycopg2.connect(
#    dbname=db_name, 
#    user="postgres", 
#    password= password, 
#    host="localhost", 
#    port="5432"
#)
#
#cur = conn.cursor()

In [ ]:
#cur.execute("""CREATE TABLE event (
#                    id SERIAL PRIMARY KEY,
#                    date DATE NOT NULL,
#                    artist VARCHAR(512),
#                    venue VARCHAR(512),
#                    start_time TIME,
#                    end_time TIME
#);""")

In [ ]:
#conn.commit()

In [ ]:
#cur.close()
#conn.close()

In [ ]:
#load table events
#conn = psycopg2.connect(
#    dbname=db_name, 
#    user="postgres", 
#    password= password, 
#    host="localhost", 
#    port="5432"
#)
#
#cur = conn.cursor()

In [ ]:
#csv_file = "./events.csv"
#with open(csv_file, 'r', encoding='utf-8') as f:
#    cur.copy_expert("""COPY taxi(date, artist, venue, start_time, end_time) 
#                        FROM STDIN WITH CSV HEADER""", f)
#print(f"Učitani podaci iz: {csv_file}")

In [ ]:
#conn.commit()

In [ ]:
#cur.close()
#conn.close()